In [1]:
import os
import time
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from codecarbon import EmissionsTracker
from ptflops import get_model_complexity_info
import pandas as pd

In [2]:
# ----------------------------
# Utility Functions
# ----------------------------
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)


def validate(model, val_loader, criterion):
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return val_loss / total, accuracy


In [3]:
# ----------------------------
# Utility Functions
# ----------------------------
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)


def validate(model, val_loader, criterion):
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return val_loss / total, accuracy


In [4]:
# ----------------------------
# Dataset and Transform
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset   = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [00:13<00:00, 12.2MB/s]


In [5]:
# ----------------------------
# MobileNetV3-Small
# ----------------------------
class HeavyDeepMobileNetV3_Small(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        base = models.mobilenet_v3_small(weights='IMAGENET1K_V1')

        # convolutional blocks 
        extra_blocks = nn.Sequential(
            nn.Conv2d(576, 576, 3, padding=1, groups=576, bias=False),
            nn.BatchNorm2d(576),
            nn.ReLU(inplace=True),
            nn.Conv2d(576, 576, 1, bias=False),
            nn.BatchNorm2d(576),
            nn.ReLU(inplace=True),

            nn.Conv2d(576, 576, 3, padding=1, groups=576, bias=False),
            nn.BatchNorm2d(576),
            nn.ReLU(inplace=True),
            nn.Conv2d(576, 576, 1, bias=False),
            nn.BatchNorm2d(576),
            nn.ReLU(inplace=True),
        )

        # Attach deeper block at the end
        base.features = nn.Sequential(
            *list(base.features.children()),
            extra_blocks,
        )

        # Replace classifier head
        in_features = base.classifier[3].in_features
        base.classifier[3] = nn.Linear(in_features, num_classes)
        self.model = base

        # Unfreeze all layers
        for param in self.model.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.model(x)


In [6]:
# ----------------------------
# Main Training Loop
# ----------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_classes = 10
results = []

output_dir = "./emissions_mobilenetv3_small_heavy"
os.makedirs(output_dir, exist_ok=True)

for seed in [0, 1, 2, 3, 4]:
    print(f"\n🌱 Running Seed {seed}")
    set_seed(seed)

    model = HeavyDeepMobileNetV3_Small(num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # FLOPs Calculation
    macs, params = get_model_complexity_info(model, (3, 32, 32),
                                             as_strings=True,
                                             print_per_layer_stat=False,
                                             verbose=False)
    macs_value = macs.replace('Mac', '').strip()
    if 'G' in macs_value:
        macs_num = float(macs_value.replace('G', '')) * 2
        flops_per_forward = f"{macs_num:.2f} GFLOPs"
    elif 'M' in macs_value:
        macs_num = float(macs_value.replace('M', '')) / 1000 * 2
        flops_per_forward = f"{macs_num:.3f} GFLOPs"
    else:
        flops_per_forward = f"{macs_value} MACs"
    print(f"Seed {seed} → FLOPs per forward pass: {flops_per_forward} | Params: {params}")

    tracker = EmissionsTracker(
        project_name=f"MobileNetV3_Small_Heavy_Seed{seed}",
        output_dir=output_dir,
        measure_power_secs=1
    )
    tracker.start()
    start_time = time.time()

    for epoch in range(4):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = validate(model, val_loader, criterion)
        print(f"Seed {seed} | Epoch {epoch+1}/4 | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")

    emissions = tracker.stop()
    elapsed = time.time() - start_time

    # Use CodeCarbon's measured energy directly
    energy_kwh = emissions  # no manual scaling

    results.append((seed, emissions, energy_kwh, elapsed, flops_per_forward))
    print(f"🌍 Seed {seed} → CO₂: {emissions:.6f} kg | Energy: {energy_kwh:.6f} kWh | Time: {elapsed/60:.2f} min | FLOPs: {flops_per_forward}")



🌱 Running Seed 0
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 70.4MB/s]
[codecarbon WARNING @ 06:31:30] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 06:31:30] [setup] RAM Tracking...
[codecarbon INFO @ 06:31:30] [setup] CPU Tracking...


Seed 0 → FLOPs per forward pass: 0.006 GFLOPs | Params: 2.21 M


[codecarbon WARNING @ 06:31:31] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 06:31:31] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 06:31:31] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 06:31:31] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 06:31:31] [setup] GPU Tracking...
[codecarbon INFO @ 06:31:31] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 06:31:31] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 06:31:31] >>> Tracker's metadata:
[codecarbon INFO @ 06:31:31]   Platform sys

Seed 0 | Epoch 1/4 | Train Loss: 0.4865 | Val Acc: 91.56%


...

Seed 4 | Epoch 2/4 | Train Loss: 0.1786 | Val Acc: 93.15%


[codecarbon INFO @ 07:26:04] Energy consumed for RAM : 0.001007 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 07:26:04] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 07:26:04] Energy consumed for All CPU : 0.004289 kWh
[codecarbon INFO @ 07:26:04] Energy consumed for all GPUs : 0.005243 kWh. Total GPU Power : 41.4356302330959 W
[codecarbon INFO @ 07:26:04] 0.010539 kWh of electricity used since the beginning.
[codecarbon INFO @ 07:26:04] 0.011902 g.CO2eq/s mean an estimation of 375.32926592183316 kg.CO2eq/year
[codecarbon INFO @ 07:26:05] Energy consumed for RAM : 0.001010 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 07:26:05] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 07:26:05] Energy consumed for All CPU : 0.004301 kWh
[codecarbon INFO @ 07:26:05] Energy consumed for all GPUs : 0.005257 kWh. Total GPU Power : 51.078322116308804 W
[codecarbon INFO @ 07:26:05] 0.010568 kWh of electricity used 

Seed 4 | Epoch 3/4 | Train Loss: 0.1114 | Val Acc: 93.22%


[codecarbon INFO @ 07:29:08] Energy consumed for RAM : 0.001511 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 07:29:08] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 07:29:08] Energy consumed for All CPU : 0.006433 kWh
[codecarbon INFO @ 07:29:08] Energy consumed for all GPUs : 0.007863 kWh. Total GPU Power : 43.87894261424508 W
[codecarbon INFO @ 07:29:08] 0.015807 kWh of electricity used since the beginning.
[codecarbon INFO @ 07:29:08] 0.011895 g.CO2eq/s mean an estimation of 375.12063261810965 kg.CO2eq/year
[codecarbon INFO @ 07:29:09] Energy consumed for RAM : 0.001514 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 07:29:09] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 07:29:09] Energy consumed for All CPU : 0.006445 kWh
[codecarbon INFO @ 07:29:09] Energy consumed for all GPUs : 0.007878 kWh. Total GPU Power : 51.16627623382294 W
[codecarbon INFO @ 07:29:09] 0.015836 kWh of electricity used 

Seed 4 | Epoch 4/4 | Train Loss: 0.0711 | Val Acc: 93.80%
🌍 Seed 4 → CO₂: 0.009915 kg | Energy: 0.009915 kWh | Time: 12.27 min | FLOPs: 0.006 GFLOPs


In [12]:
import pandas as pd
import os

output_dir = "./emissions_mobilenetv3_small_heavy"  # same as used during training
csv_file = os.path.join(output_dir, "emissions.csv")

# Read the CodeCarbon CSV
df = pd.read_csv(csv_file)

# Prepare results with energy
results_with_energy = []

for seed, emissions, energy_kwh_from_results, elapsed, flops in results:
    project_name = f"MobileNetV3_Small_Heavy_Seed{seed}"

    # Fetch the last entry for that seed to get the total energy consumed
    row = df[df['project_name'] == project_name].iloc[-1]
    energy_consumed_total = row['energy_consumed']  # kWh

    results_with_energy.append((seed, emissions, elapsed, flops, energy_consumed_total))

# Print results summary including energy
print("\n--- Results Summary ---")
for seed, emissions, elapsed, flops, energy in results_with_energy:
    print(f"Seed {seed}: CO₂ = {emissions:.6f} kg, Time = {elapsed/60:.2f} min, FLOPs = {flops}, Energy = {energy:.6f} kWh")


--- Results Summary ---
Seed 0: CO₂ = 0.009586 kg, Time = 11.94 min, FLOPs = 0.006 GFLOPs, Energy = 0.020362 kWh
Seed 1: CO₂ = 0.009746 kg, Time = 11.98 min, FLOPs = 0.006 GFLOPs, Energy = 0.020701 kWh
Seed 2: CO₂ = 0.009805 kg, Time = 12.14 min, FLOPs = 0.006 GFLOPs, Energy = 0.020826 kWh
Seed 3: CO₂ = 0.009870 kg, Time = 12.23 min, FLOPs = 0.006 GFLOPs, Energy = 0.020965 kWh
Seed 4: CO₂ = 0.009915 kg, Time = 12.27 min, FLOPs = 0.006 GFLOPs, Energy = 0.021061 kWh


In [13]:
import numpy as np

# Convert results_with_energy to a NumPy array for easier calculation
results_array = np.array(results_with_energy, dtype=object)

# Extract columns for calculation (excluding seed and flops which are constant)
# Columns: seed (0), emissions (1), elapsed (2), flops (3), energy (4)
emissions_values = results_array[:, 1].astype(float)
elapsed_values = results_array[:, 2].astype(float)
energy_values = results_array[:, 4].astype(float)

# Calculate mean and standard deviation
mean_emissions = np.mean(emissions_values)
std_emissions = np.std(emissions_values)

mean_elapsed = np.mean(elapsed_values)
std_elapsed = np.std(elapsed_values)

mean_energy = np.mean(energy_values)
std_energy = np.std(energy_values)


print("\n--- Mean and Standard Deviation ---")
print(f"CO₂ Emissions (kg): Mean = {mean_emissions:.6f}, Std Dev = {std_emissions:.6f}")
print(f"Time (min): Mean = {mean_elapsed/60:.2f}, Std Dev = {std_elapsed/60:.2f}")
print(f"Energy (kWh): Mean = {mean_energy:.6f}, Std Dev = {std_energy:.6f}")


--- Mean and Standard Deviation ---
CO₂ Emissions (kg): Mean = 0.009784, Std Dev = 0.000114
Time (min): Mean = 12.11, Std Dev = 0.13
Energy (kWh): Mean = 0.020783, Std Dev = 0.000243
